In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
import numpy as np
from qiskit_aer import AerSimulator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.library import QAOAAnsatz
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler, EstimatorV2 as Estimator
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [12]:
# Initialize Quadratic Program
mod = QuadraticProgram("maximum coverage problem")

# Define binary variables
for i in range(4):
    mod.binary_var(name="x" + str(i + 1))

# Define the linear term for the objective
lin = np.ones(4)
quad = np.zeros((4, 4))

# Add the original objective: Minimize the sum of x_i
mod.minimize(constant=0, linear=lin, quadratic=quad)

# Adjust penalty weights
alpha = 10  # High penalty for violations (x_i + x_j < 1)
beta = 2    # Low penalty for excessive satisfaction (x_i + x_j > 1)

penalties = [
    ("x1", "x2"),
    ("x1", "x3"),
    ("x1", "x4"),
    ("x2", "x4"),
    ("x3", "x4"),
]

for var1, var2 in penalties:
    # Penalty for under-satisfaction: alpha * (1 - x_i - x_j)^2
    mod.objective.constant += alpha  # Constant term (from 1)
    mod.objective.linear[var1] += -alpha  # Linear term for x_i
    mod.objective.linear[var2] += -alpha  # Linear term for x_j
    mod.objective.quadratic[var1, var2] += alpha  # Quadratic term for x_i * x_j

    # Penalty for over-satisfaction: -gamma * x_i * x_j
    #mod.objective.quadratic[var1, var2] -= beta

# Display the problem
print(mod.prettyprint())

Problem name: maximum coverage problem

Minimize
  10*x1*x2 + 10*x1*x3 + 10*x1*x4 + 10*x2*x4 + 10*x3*x4 - 29*x1 - 19*x2 - 19*x3
  - 29*x4 + 50

Subject to
  No constraints

  Binary variables (4)
    x1 x2 x3 x4



In [13]:
penalty_value = 1
qubo_problem = QuadraticProgramToQubo(penalty=penalty_value).convert(mod)

In [14]:
print(qubo_problem.prettyprint())

Problem name: maximum coverage problem

Minimize
  10*x1*x2 + 10*x1*x3 + 10*x1*x4 + 10*x2*x4 + 10*x3*x4 - 29*x1 - 19*x2 - 19*x3
  - 29*x4 + 50

Subject to
  No constraints

  Binary variables (4)
    x1 x2 x3 x4



In [20]:
values = {'x1': 1, 'x2': 1, 'x3': 0, 'x4': 0}
qubo_problem.objective.evaluate([values[f'x{i}'] for i in range(1, 5)])

12.0